In [25]:
import jpy_tools.parseSnake as jps

In [26]:
configPath = '/public/home/liuzj/publicPipeline/snuupy/snakemake/config_kb.yaml'
snakePath = '/public/home/liuzj/publicPipeline/snuupy/snakemake/snakefile_kb'

In [27]:
snakeFile = jps.SnakeMakeFile()

In [28]:
snakeHeader = jps.SnakeHeader(snakeFile, configPath)
snakeHeader.generateContent()

config contents:

    pipelineDir     :/public/home/liuzj/publicPipeline/snuupy/

     resultDir      :/public/home/liuzj/publicPipeline/snuupy/example/snakemakeResults/

   rawNanoporeFa    :/public/home/liuzj/publicPipeline/snuupy/example/00_data/nanopore.fa

 nanoporeWorkspace  :/public/home/liuzj/publicPipeline/snuupy/example/00_data/workspace

 nanoporeSeqSummary :/public/home/liuzj/publicPipeline/snuupy/example/00_data/nanoporeSeqSummary.txt

    geneAnnoBed     :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/gene.bed

  geneAnnoRepreBed  :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/araport11.representative.gene_model.bed

    geneNot12Bed    :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/geneNotBed12.bed

      genomeFa      :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/genome.fa

     genomeFai      :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/genome.fa.fa

In [29]:
runCellRanger = jps.SnakeRule(snakeFile, 'runCellRanger', 1, 2)
runCellRanger.setInput(b=['runCellRangerConfig'])
runCellRanger.setOutput(a = ['runCellRangerFinished.empty'])
runCellRanger.setShell("""
python ./removeExonRegion/removeExonRegion.py runCellRanger -p {input.runCellRangerConfig} &&\
touch {output.runCellRangerFinished}
""")
runCellRanger.generateContent()

In [30]:
parseIllumina = jps.SnakeRule(snakeFile, 'parseIllumina', 2, 2)
parseIllumina.setInput(a = ['runCellRangerFinished.empty'])
parseIllumina.setOutput(a = ['parseIlluminaResults.index'])
parseIllumina.setParams(b = ['genomeFai'], c = dict(useBarcodeGz = 'f"{config[\'resultDir\']}step1_runCellRanger/test/outs/filtered_feature_bc_matrix/barcodes.tsv.gz"', useBarcode = 'f"{config[\'resultDir\']}step1_runCellRanger/test/outs/filtered_feature_bc_matrix/barcodes.tsv"', runCellRangerBam = 'f"{config[\'resultDir\']}step1_runCellRanger/test/outs/possorted_genome_bam.bam"',windowSize=500))
parseIllumina.setShell("""
gzip -d -c {params.useBarcodeGz} > {params.useBarcode} &&
python ./snuupy/snuupy.py parseIllumina --bam {params.runCellRangerBam} --barcode {params.useBarcode} --genome {params.genomeFai} --window {params.windowSize} --parsed {output.parseIlluminaResults}
""")
parseIllumina.generateContent()

In [31]:
minimapMappingRaw = jps.SnakeRule(snakeFile, 'minimapMappingRaw', 3,2)
minimapMappingRaw.setInput(b = ['rawNanoporeFa'])
minimapMappingRaw.setOutput(a=['rawMappingResult.bam'])
minimapMappingRaw.setParams(b = ['minimap2Path', 'genomeFa', 'geneAnnoBed'])
minimapMappingRaw.setShell("""
{params.minimap2Path} -ax splice --secondary=no -uf --MD --sam-hit-only -t {threads} --junc-bed {params.geneAnnoBed} {params.genomeFa} {input.rawNanoporeFa} | samtools sort - -o {output.rawMappingResult} && samtools index {output.rawMappingResult}
""")
minimapMappingRaw.generateContent()

In [32]:
addUnmappedBaseTag = jps.SnakeRule(snakeFile,'addUnmappedBaseTag', 4, 1)
addUnmappedBaseTag.setInput(a=['rawMappingResult'])
addUnmappedBaseTag.setOutput(a=['rawMappingResultAddUnmappedTag.bam'])
addUnmappedBaseTag.setParams(b=['rawNanoporeFa'])
addUnmappedBaseTag.setShell("""
python ./snuupy/snuupy.py addUnmappedBaseTag -i {input.rawMappingResult} -f {params.rawNanoporeFa} -o {output.rawMappingResultAddUnmappedTag}
""")
addUnmappedBaseTag.generateContent()

In [33]:
generateNanoporeWindow = jps.SnakeRule(snakeFile, 'generateNanoporeWindow', 5, 5)
generateNanoporeWindow.setInput(a=['rawMappingResultAddUnmappedTag'])
generateNanoporeWindow.setOutput(a=['generateNanoporeWindowFinished.empty'])
generateNanoporeWindow.setParams(a=['generateNanoporeWindow/'],b=['genomeFai'], c=dict(windowSize=500))
generateNanoporeWindow.setShell("""
python ./snuupy/snuupy.py generateNanoporeWindow --genome {params.genomeFai} -b {input.rawMappingResultAddUnmappedTag} -w {params.windowSize} -o {params.generateNanoporeWindow} && touch {output.generateNanoporeWindowFinished}
""")
generateNanoporeWindow.generateContent()

In [34]:
generateIlluminaWindow = jps.SnakeRule(snakeFile, 'generateIlluminaWindow', 6, 5)
generateIlluminaWindow.setInput(a=['parseIlluminaResults'])
generateIlluminaWindow.setOutput(a=['generateIlluminaWindowFinished.empty'])
generateIlluminaWindow.setParams(a=['generateIlluminaWindow/'])
generateIlluminaWindow.setShell("""
python ./snuupy/snuupy.py generateIlluminaWindow -i {input.parseIlluminaResults} -o {params.generateIlluminaWindow} && touch {output.generateIlluminaWindowFinished}
""")
generateIlluminaWindow.generateContent()

In [35]:
windowBlast = jps.SnakeRule(snakeFile, 'windowBlast', 7, 2)
windowBlast.setInput(a=['generateIlluminaWindowFinished', 'generateNanoporeWindowFinished'])
windowBlast.setOutput(a=['windowBlastFinished.empty'])
windowBlast.setParams(a=['blastResults/', 'generateIlluminaWindow/', 'generateNanoporeWindow/'], b=['blastPath'])
windowBlast.setShell("""
python ./snuupy/snuupy.py windowBlast -i {params.generateIlluminaWindow} -n {params.generateNanoporeWindow} -o {params.blastResults} -t {threads} -b {params.blastPath} && touch {output.windowBlastFinished}
""")
windowBlast.generateContent()

In [36]:
getMismatch = jps.SnakeRule(snakeFile, 'getMismatch', 8, 2)
getMismatch.setInput(a=['windowBlastFinished', 'rawMappingResultAddUnmappedTag'])
getMismatch.setOutput(a=['nanoporeED.feather'])
getMismatch.setParams(c=dict(windowBlastMerged = 'f"{config[\'resultDir\']}step9_windowBlast/blastResults/allResult.result"'))
getMismatch.setShell("""
python ./snuupy/snuupy.py getMismatch -i {params.windowBlastMerged} -b {input.rawMappingResultAddUnmappedTag} -o {output.nanoporeED} -t {threads}
""")
getMismatch.generateContent()

In [37]:
barcodeAssignment = jps.SnakeRule(snakeFile, 'barcodeAssignment', 9, 2)
barcodeAssignment.setInput(a=['nanoporeED'])
barcodeAssignment.setOutput(a=['nanoporeAssigned.feather'])
barcodeAssignment.setParams(c=dict(barcodeED=3, umiED=3))
barcodeAssignment.setShell("""
python ./snuupy/snuupy.py barcodeAssignment -i {input.nanoporeED} -o {output.nanoporeAssigned} --ED-barcode {params.barcodeED} --ED-UMI {params.umiED}
""")
barcodeAssignment.generateContent()

In [38]:
polishReads = jps.SnakeRule(snakeFile, 'polishReads', 10, 2)
polishReads.setInput(a=['nanoporeAssigned'], b=['rawNanoporeFa'])
polishReads.setOutput(a=['polishedReads.fa'])
polishReads.setParams(a=['polishReadsTempRaw/', 'polishReadsTempFinal/'], b=['minimap2Path','seqkitPath', 'poaPath', 'raconPath', 'penaltyMatPath'])
polishReads.setShell("""
python ./snuupy/snuupy.py polishReads -i {input.nanoporeAssigned} -f {input.rawNanoporeFa} -T {params.polishReadsTempRaw} -F {params.polishReadsTempFinal} -o {output.polishedReads} -p {params.penaltyMatPath} -t {threads} --minimap2 {params.minimap2Path} --poa {params.poaPath} --racon {params.raconPath} --seqkit {params.seqkitPath}
""")
polishReads.generateContent()



In [39]:
minimapMappingPolished = jps.SnakeRule(snakeFile, 'minimapMappingPolished', 11,2)
minimapMappingPolished.setInput(a = ['polishedReads'])
minimapMappingPolished.setOutput(a=['polishedMappingResult.bam'])
minimapMappingPolished.setParams(b = ['minimap2Path', 'genomeFa', 'geneAnnoBed'])
minimapMappingPolished.setShell("""
{params.minimap2Path} -ax splice --secondary=no -uf --MD --sam-hit-only -t {threads} --junc-bed {params.geneAnnoBed} {params.genomeFa} {input.polishedReads} | samtools sort - -o {output.polishedMappingResult} && samtools index {output.polishedMappingResult}
""")
minimapMappingPolished.generateContent()

In [40]:
addGeneName = jps.SnakeRule(snakeFile, 'addGeneName', 12, 1)
addGeneName.setInput(a=['polishedMappingResult'])
addGeneName.setOutput(a=['polishedReadsAddGNPickle.pickle', 'polishedReadsAddGNBam.bam'])
addGeneName.setParams(b=['geneAnnoBed', 'bedtoolsPath'])
addGeneName.setShell("""
python ./snuupy/snuupy.py addGeneName -i {input.polishedMappingResult} --bed {params.geneAnnoBed} --out-pickle {output.polishedReadsAddGNPickle} --out-bam {output.polishedReadsAddGNBam} --bedtools {params.bedtoolsPath}
""")
addGeneName.generateContent()

In [41]:
getSpliceInfo = jps.SnakeRule(snakeFile, 'getSpliceInfo', 13, 1)
getSpliceInfo.setInput(a=['polishedMappingResult', 'polishedReadsAddGNPickle'])
getSpliceInfo.setOutput(a=['splicingInfo.tsv'])
getSpliceInfo.setParams(b=['bedtoolsPath', 'geneAnnoRepreBed'])
getSpliceInfo.setShell("""
python ./snuupy/snuupy.py getSpliceInfo -i {input.polishedMappingResult} -b {params.geneAnnoRepreBed} -o {output.splicingInfo} -g {input.polishedReadsAddGNPickle} --bedtools {params.bedtoolsPath}
""")
getSpliceInfo.generateContent()

In [42]:
addPolyATag = jps.SnakeRule(snakeFile, 'addPolyATag', 14, 2)
addPolyATag.setInput(a=['nanoporeAssigned', 'polishedReadsAddGNBam'], b=['rawNanoporeFa', 'nanoporeWorkspace', 'nanoporeSeqSummary', 'geneAnnoBed'])
addPolyATag.setOutput(a=['polishedReadsAddGNPABam.bam'])
addPolyATag.setParams(a=['polyACallerTemp/'] ,b=['genomeFa', 'geneAnnoBed','minimap2Path'])
addPolyATag.setShell("""
    python ./snuupy/snuupy.py addPolyATag --in-fasta {input.rawNanoporeFa} --genome {params.genomeFa} -t {threads} --in-f5-workspace {input.nanoporeWorkspace} --in-f5-summary {input.nanoporeSeqSummary} --bed {params.geneAnnoBed} --tempDir {params.polyACallerTemp} --feather {input.nanoporeAssigned} --in-bam {input.polishedReadsAddGNBam} --out-bam {output.polishedReadsAddGNPABam} --minimap {params.minimap2Path}
""")
addPolyATag.generateContent()

In [43]:
polyAClusterDetected = jps.SnakeRule(snakeFile, 'polyAClusterDetected', 15, 2)
polyAClusterDetected.setInput(a=['polishedReadsAddGNPABam'])
polyAClusterDetected.setOutput(a=['polyAClusterDetectedFinished.empty'])
polyAClusterDetected.setParams(a=['polyACluster/'], b=['geneNot12Bed','genomeFa'])
polyAClusterDetected.setShell("""
python ./snuupy/snuupy.py polyAClusterDetected --infile {input.polishedReadsAddGNPABam} --gene-bed {params.geneNot12Bed} --out-dir {params.polyACluster} -t {threads} --fasta {params.genomeFa} && touch {output.polyAClusterDetectedFinished}
""")
polyAClusterDetected.generateContent()

In [44]:
generateMtx = jps.SnakeRule(snakeFile, 'generateMtx', 16)
generateMtx.setInput(a=['splicingInfo', 'polishedReadsAddGNPABam', 'polyAClusterDetectedFinished'])
generateMtx.setOutput(a=['generateMtxFinished.empty'])
generateMtx.setParams(a=['IlluminaMultiMat/', 'NanoporeMultiMat/'], b=['usedIntron'] ,c=dict(illuminaExpressionMtx='f"{config[\'resultDir\']}step3_rerunCellRanger/test/outs/filtered_feature_bc_matrix.h5"', apaCluster='f"{config[\'resultDir\']}step17_polyAClusterDetected/polyACluster/polya_cluster.filtered.bed"'))
generateMtx.setShell("""
python ./snuupy/snuupy.py generateMtx -i {input.splicingInfo} --in-illumina {params.illuminaExpressionMtx} --apa-pac {params.apaCluster} --apa-bam {input.polishedReadsAddGNPABam} --ir --ir-list {params.usedIntron} --out-nanopore {params.NanoporeMultiMat} --out-illumina {params.IlluminaMultiMat} && touch {output.generateMtxFinished}
""")
generateMtx.generateContent()

In [45]:
snakeAll = jps.SnakeAll(snakeFile)
snakeAll.generateContent(generateMtxFinished=0)

In [46]:
snakeFile.generateContent(snakePath)

configfile: "/public/home/liuzj/publicPipeline/snuupy/snakemake/config.yaml"
pipelineDir = config['pipelineDir']


rule all:
    input:
        generateMtxFinished = f"{config['resultDir']}step16_generateMtx/generateMtxFinished.empty"

rule runCellRanger:
    input:
        runCellRangerConfig = config['runCellRangerConfig']
    output:
        runCellRangerFinished = f"{config['resultDir']}step1_runCellRanger/runCellRangerFinished.empty"
    params:
        gpu = "0"

    threads:2
    shell:
        """
cd {pipelineDir}
python ./optimizedCellRanger/optimizedCellRanger.py runCellRanger -p {input.runCellRangerConfig} &&touch {output.runCellRangerFinished}
        """

rule parseIllumina:
    input:
        runCellRangerFinished = f"{config['resultDir']}step1_runCellRanger/runCellRangerFinished.empty"
    output:
        parseIlluminaResults = f"{config['resultDir']}step2_parseIllumina/parseIlluminaResults.index"
    params:
        genomeFai = config['genomeFai'],
        useBarcodeGz 